In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import round, col, to_date
from pyspark.sql.functions import year, avg, round, col
import time

# Create a SparkSession
spark = SparkSession.builder.appName("HomeSalesAnalysis").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
house_sales_df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

# Show DataFrame
house_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
from pyspark.sql import SparkSession
house_sales_df.createOrReplaceTempView('home_sales')


In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
filtered_houses_df_1 = house_sales_df.filter((col("bedrooms") == 4))

# Group by date_built (which is an INT representing the year) and calculate the average price
avg_price_4br_year = filtered_houses_df_1.groupBy("date_built") \
    .agg(round(avg(col("price")), 2).alias("average_price")) \
    .orderBy("date_built")

# Show the results
print("Average price of 4 bedroom house sold per year")
avg_price_4br_year.show(truncate=False)

Average price of 4 bedroom house sold per year
+----------+-------------+
|date_built|average_price|
+----------+-------------+
|2010      |296800.75    |
|2011      |302141.9     |
|2012      |298233.42    |
|2013      |299999.39    |
|2014      |299073.89    |
|2015      |307908.86    |
|2016      |296050.24    |
|2017      |296576.69    |
+----------+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
filtered_houses_df_2 = house_sales_df.filter((col("bedrooms") == 3) & (col("bathrooms") == 3))

# Group by date_built (which is an INT representing the year) and calculate the average price
avg_price_3br_3ba_year = filtered_houses_df_2.groupBy("date_built") \
    .agg(round(avg(col("price")), 2).alias("average_price")) \
    .orderBy("date_built")

# Show the results
print("Average price of 3 bedroom, 3 bathroom house sold per year")
avg_price_3br_3ba_year.show(truncate=False)

Average price of 3 bedroom, 3 bathroom house sold per year
+----------+-------------+
|date_built|average_price|
+----------+-------------+
|2010      |292859.62    |
|2011      |291117.47    |
|2012      |293683.19    |
|2013      |295962.27    |
|2014      |290852.27    |
|2015      |288770.3     |
|2016      |290555.07    |
|2017      |292676.79    |
+----------+-------------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
# Filter the DataFrame for homes with 3 bedrooms, 3 bathrooms, 2 floors, and >= 2000 square feet
filtered_houses_df_3 = house_sales_df.filter(
    (col("bedrooms") == 3) &
    (col("bathrooms") == 3) &
    (col("floors") == 2) &
    (col("sqft_living") >= 2000)
)

# Group by date_built (which is an INT representing the year) and calculate the average price
avg_price_2fl_2000sqft_year = filtered_houses_df_3.groupBy("date_built") \
    .agg(round(avg(col("price")), 2).alias("average_price")) \
    .orderBy("date_built")

# Show the results
print("Average price of 3 bedroom, 3 bathroom, 2 floor and 2,000+ sqft house sold per year")
avg_price_2fl_2000sqft_year.show(truncate=False)


Average price of 3 bedroom, 3 bathroom, 2 floor and 2,000+ sqft house sold per year
+----------+-------------+
|date_built|average_price|
+----------+-------------+
|2010      |285010.22    |
|2011      |276553.81    |
|2012      |307539.97    |
|2013      |303676.79    |
|2014      |298264.72    |
|2015      |297609.97    |
|2016      |293965.1     |
|2017      |280317.58    |
+----------+-------------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

# Group by "view" rating and calculate the average price
avg_price_per_view = house_sales_df.groupBy("view") \
    .agg(round(avg(col("price")), 2).alias("average_price")) \
    .filter(col("average_price") >= 350000) \
    .orderBy(col("view").desc())

# Show the results
print("Average price of homes per view rating (>= $350,000):")
avg_price_per_view.show(truncate=False)

# Calculate and print the run time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

Average price of homes per view rating (>= $350,000):
+----+-------------+
|view|average_price|
+----+-------------+
|100 |1026669.5    |
|99  |1061201.42   |
|98  |1053739.33   |
|97  |1129040.15   |
|96  |1017815.92   |
|95  |1054325.6    |
|94  |1033536.2    |
|93  |1026006.06   |
|92  |970402.55    |
|91  |1137372.73   |
|90  |1062654.16   |
|89  |1107839.15   |
|88  |1031719.35   |
|87  |1072285.2    |
|86  |1070444.25   |
|85  |1056336.74   |
|84  |1117233.13   |
|83  |1033965.93   |
|82  |1063498.0    |
|81  |1053472.79   |
+----+-------------+
only showing top 20 rows

--- 9.202957153320312e-05 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

# Group by "view" rating and calculate the average price
avg_price_per_view = house_sales_df.groupBy("view") \
    .agg(round(avg(col("price")), 2).alias("average_price")) \
    .filter(col("average_price") >= 350000) \
    .orderBy(col("view").desc())

# Show the results
print("Average price of homes per view rating (>= $350,000):")
avg_price_per_view.show(truncate=False)

# Calculate and print the run time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

Average price of homes per view rating (>= $350,000):
+----+-------------+
|view|average_price|
+----+-------------+
|100 |1026669.5    |
|99  |1061201.42   |
|98  |1053739.33   |
|97  |1129040.15   |
|96  |1017815.92   |
|95  |1054325.6    |
|94  |1033536.2    |
|93  |1026006.06   |
|92  |970402.55    |
|91  |1137372.73   |
|90  |1062654.16   |
|89  |1107839.15   |
|88  |1031719.35   |
|87  |1072285.2    |
|86  |1070444.25   |
|85  |1056336.74   |
|84  |1117233.13   |
|83  |1033965.93   |
|82  |1063498.0    |
|81  |1053472.79   |
+----+-------------+
only showing top 20 rows

--- 0.00011563301086425781 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
parquet_df = house_sales_df.write.mode("overwrite").partitionBy("date_built").parquet("path/to/save/partitioned_home_sales")

In [17]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("path/to/save/partitioned_home_sales")

In [18]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet')

In [19]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

avg_price_per_view = parquet_df.groupBy("view") \
    .agg(round(avg(col("price")), 2).alias("average_price")) \
    .filter(col("average_price") >= 350000) \
    .orderBy(col("view").desc())

# Show the results
print("Average price of homes per view rating (>= $350,000):")
avg_price_per_view.show(truncate=False)

# Calculate and print the run time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

Average price of homes per view rating (>= $350,000):
+----+-------------+
|view|average_price|
+----+-------------+
|100 |1026669.5    |
|99  |1061201.42   |
|98  |1053739.33   |
|97  |1129040.15   |
|96  |1017815.92   |
|95  |1054325.6    |
|94  |1033536.2    |
|93  |1026006.06   |
|92  |970402.55    |
|91  |1137372.73   |
|90  |1062654.16   |
|89  |1107839.15   |
|88  |1031719.35   |
|87  |1072285.2    |
|86  |1070444.25   |
|85  |1056336.74   |
|84  |1117233.13   |
|83  |1033965.93   |
|82  |1063498.0    |
|81  |1053472.79   |
+----+-------------+
only showing top 20 rows

--- 0.000118255615234375 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [21]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False